In [1]:
search_for = 138
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.135650634765625
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 1097396311
type: [1, 1, 1, 1, 138] 138
cases of this type: 362673936
100000 50.18954450388526
200000 100.35127293015013
300000 105.81855574754339
400000 106.40092393653445
500000 105.93622698532138
600000 104.22353685183008
700000 107.57791571142423
800000 103.38323351426688
900000 102.4609762526438
1000000 102.1900468973359
1100000 102.46646559356677
1200000 102.31370049460138
1300000 83.5830996461269
1400000 40.757839117869544
1500000 100.86829131102027
1600000 101.32557598130822
1700000 101.75696337858645
1800000 100.29175792663733
1900000 101.35216980628478
2000000 105.89720323812554
2100000 102.2537598721762
2200000 101.15196624614319
2300000 101.7056496634585
2400000 101.4579272021753
2500000 102.42342024774904
2600000 97.02570087822399
2700000 24.44741783377754
2800000 98.08192184181003
2900000 100.40468109215533
3000000 98

29400000 116.81658083571702
29500000 114.28940188470919
29600000 113.17686096326392
29700000 115.45827600858748
29800000 113.68151961947169
29900000 113.06895426602993
30000000 109.44347536845393
30100000 110.29189917182114
30200000 95.84154335813297
30300000 35.99443374956984
30400000 106.44080304626037
30500000 110.02867616169443
30600000 109.39907002494631
30700000 122.63226922771803
30800000 114.28389633338375
30900000 110.19972984879622
31000000 113.59706607809167
31100000 116.63777073571133
31200000 112.56886608683389
31300000 107.20542026283799
31400000 45.72372729217751
31500000 117.7027063734659
31600000 115.73620509463686
31700000 110.12927759728021
31800000 52.54349489503633
31900000 93.14113038621686
32000000 109.90368814188743
32100000 108.5884423021304
32200000 110.32047685754783
32300000 104.60537475225985
32400000 103.27302787079606
32500000 108.40322145941788
32600000 104.20619651935506
32700000 104.88029371343417
32800000 102.3772032625261
32900000 26.988285548523034


59100000 87.73441086400632
59200000 22.452413242751764
59300000 93.85872128003977
59400000 93.04408456430247
59500000 93.12744643442217
59600000 94.44277571062874
59700000 96.13908467366926
59800000 96.73235861875467
59900000 98.45528467126339
60000000 98.02721352904973
60100000 42.312061440944504
60200000 102.21647312765354
60300000 100.56766803437576
60400000 101.7531128600787
60500000 103.46125905783005
60600000 108.33756015966928
60700000 105.1397156924927
60800000 105.57558335475379
60900000 63.91960724630055
61000000 77.4569429185795
61100000 104.63427303029019
61200000 99.78877633564522
61300000 101.29608162782448
61400000 109.35498091053478
61500000 96.9093587134912
61600000 93.39012182327903
61700000 94.55089443684224
61800000 32.49583022950251
61900000 87.32658643266926
62000000 94.12881148747782
62100000 93.92549641991042
62200000 100.89647647431862
62300000 97.35871012147174
62400000 98.48691306338691
62500000 102.27174970073835
62600000 103.78668600559645
62700000 34.51182

89200000 102.75431977513921
89300000 105.86854461079768
89400000 113.29629995747587
89500000 108.67267012161136
89600000 102.22148150191197
89700000 103.83420292697866
89800000 102.95524830382382
89900000 99.10766005033419
90000000 76.66592066419618
90100000 55.220002914428136
90200000 93.64016603838095
90300000 90.50761929631945
90400000 91.57034301051787
90500000 94.90475274741891
90600000 93.25442437275692
90700000 39.58057078963804
90800000 78.44113604954234
90900000 95.5535509795874
91000000 92.77474848734876
91100000 91.00314290387398
91200000 92.03450470186878
91300000 91.88447640680462
91400000 34.68593352762871
91500000 101.99217997005502
91600000 99.84413490593778
91700000 102.03297355844222
91800000 101.12100505755113
91900000 103.10740769783338
92000000 112.79317243687292
92100000 108.3795185015235
92200000 106.2971490757093
92300000 105.6785638718693
92400000 104.56185039199387
92500000 102.73515308418666
92600000 103.85272880905067
92700000 31.21056196620228
92800000 94.1

118700000 97.69283633051356
118800000 104.60517875894236
118900000 92.49063885399707
119000000 91.40284624254663
119100000 81.65809325901819
119200000 39.35881841199721
119300000 88.80357707157586
119400000 92.22143523852793
119500000 90.42079577812689
119600000 46.448428311251675
119700000 68.59949803768505
119800000 90.7587790059395
119900000 90.98991095821535
120000000 90.15034513815401
120100000 33.0309835787763
120200000 91.86199537812419
120300000 91.32949234457097
120400000 94.23489529976763
120500000 96.3245901747095
120600000 95.21126309260862
120700000 103.62555451597345
120800000 99.26904650677811
120900000 103.47153860555105
121000000 93.18779639664078
121100000 103.1839577395085
121200000 100.97656906887528
121300000 95.80844555597812
121400000 91.4737861560829
121500000 89.46593227635105
121600000 87.77310427725706
121700000 85.48247881904227
121800000 43.959002105686245
121900000 72.5425686121272
122000000 87.82976538966699
122100000 84.99890744765081
122200000 65.109550

...

982700000 5.069449313948177
982800000 10.505644139493324
982900000 9.473638632204471
983000000 4.786019773456253
983100000 7.389914050168128
983200000 9.518456070356102
983300000 11.081173998507795
983400000 8.423858954410653
983500000 8.135435818138761
983600000 6.635521464061683
983700000 1.8548029773721577
983800000 7.66895325787309
983900000 7.638817343115891
984000000 7.995164821046171
984100000 7.474578531042016
984200000 6.615224480185576
984300000 3.7481134286512945
984400000 3.7617883417129736
984500000 7.271519617633814
984600000 6.624134366615461
984700000 8.251605232704433
984800000 6.822586122610711
984900000 6.1216594784296126
985000000 3.83238750765844
985100000 5.32272209800491
985200000 6.829846913816048
985300000 8.19869567588561
985400000 7.174733707858225
985500000 6.939293040020457
985600000 5.4586442481013995
985700000 2.9493745307190133
985800000 6.779270099805649
985900000 7.675810369844467
986000000 7.642910040008893
986100000 6.746618367216991
986200000 6.3776

1011100000 1.9365978967606603
1011200000 1.5633752030171195
1011300000 0.5001849421248761
1011400000 2.909550129647287
1011500000 2.196610089899077
1011600000 4.036302201363405
1011700000 3.416266614550761
1011800000 3.3456908336272964
1011900000 1.5845194405027834
1012000000 2.1089710289284525
1012100000 2.167253671044185
1012200000 2.791638451700659
1012300000 2.328108959984428
1012400000 1.6690890999834942
1012500000 1.8844050296225914
1012600000 0.490685004714483
1012700000 1.7909792817057184
1012800000 2.789223682155688
1012900000 2.393494836798173
1013000000 2.3832830633002935
1013100000 2.7771333517070813
1013200000 1.3132903583916524
1013300000 0.753380920264003
1013400000 2.324747617172242
1013500000 1.5868337336619458
1013600000 2.1706627316232825
1013700000 2.4090667975249174
1013800000 1.6350193277442335
1013900000 0.47579831550951734
1014000000 1.0338250075421704
1014100000 2.1062405853547017
1014200000 1.85609162567078
1014300000 1.7311541045841068
1014400000 1.2973592170

1036800000 2.282230443292178
1036900000 1.19912202638219
1037000000 1.6506928901191362
1037100000 2.4387814904858196
1037200000 1.3390705543682815
1037300000 1.5848720276163029
1037400000 1.8354970534561714
1037500000 1.4560398951634745
1037600000 1.6065332071471619
1037700000 1.757475703615456
1037800000 1.3850482252379417
1037900000 1.6800006696594405
1038000000 1.6179036759788825
1038100000 1.3820592706042887
1038200000 1.6572742118032102
1038300000 1.7045796050110624
1038400000 1.4093098464380205
1038500000 1.6591930704146085
1038600000 1.8312889725868071
1038700000 1.4385938795500695
1038800000 1.67762540488539
1038900000 1.9148937895996603
1039000000 1.7118768412110625
1039100000 1.5431684965486254
1039200000 1.869329449828522
1039300000 1.5346853713441215
1039400000 1.3093281141651187
1039500000 1.9313035925237887
1039600000 1.7018427976148764
1039700000 1.2022826283068637
1039800000 1.8313295812563042
1039900000 1.9966649790310445
1040000000 1.6099740198352395
1040100000 1.9835

1057900000 0.9986819167333364
1058000000 0.8835853932825626
1058100000 0.9364361359251068
1058200000 1.196758606800884
1058300000 1.1546002090094316
1058400000 0.8281646500624259
1058500000 1.0981259177533567
1058600000 0.9438478057176033
1058700000 0.8564427721175127
1058800000 0.9738414586963893
1058900000 0.9882110846152405
1059000000 0.8541460449871593
1059100000 0.9716676175706348
1059200000 1.142052794417421
1059300000 0.738212596699581
1059400000 0.8690658531057299
1059500000 1.122431172353161
1059600000 0.9018832190997865
1059700000 0.731957627153498
1059800000 1.0722813488883483
1059900000 0.9316167849196374
1060000000 0.8561990802958925
1060100000 0.9323714901553505
1060200000 1.0596385150085383
1060300000 0.6749767866630971
1060400000 0.8635690471438944
1060500000 1.0525275101604745
1060600000 0.8159852991070689
1060700000 0.7196017898209022
1060800000 1.027064270008709
1060900000 0.8457687421714742
1061000000 0.7043928837408933
1061100000 0.9922767521397465
1061200000 0.989

1084500000 0.0879334772797876
1084600000 0.05630674693857232
1084700000 0.05577270895562437
best so far: 0
type: [2, 1, 3, 1, 23] 138
cases of this type: 2518569
1084800000 0.13642669536827606
1084900000 0.2112994599505961
1085000000 0.18533111244021322
1085100000 0.16884755399942664
1085200000 0.1671608371296088
1085300000 0.16788430560140413
1085400000 0.18244581593236395
1085500000 0.2239854763596773
1085600000 0.16838150174276761
1085700000 0.17950494991024268
1085800000 0.10393045621628165
1085900000 0.05782563968919384
1086000000 0.057490804510262276
1086100000 0.05689187020488779
1086200000 0.05637690033972529
1086300000 0.07352568531325326
1086400000 0.19020197180119355
1086500000 0.16141213475262786
1086600000 0.15594346972260012
1086700000 0.15239362498239875
1086800000 0.1482956572721488
1086900000 0.14479241488333477
1087000000 0.1427254671626548
1087100000 0.14469217316996588
1087200000 0.1584635281662795
best so far: 0
type: [2, 1, 3, 23, 1] 138
cases of this type: 109503